# Seitenspiegel-Analyse am DTA OCR Korpus

Ziel des vorliegenden Projekts ist es, das [DTA-Kernkorpus](https://deutschestextarchiv.de) für typographische Analysen zu nutzen. Im Spezifischen geht es um den Satzspiegel: *Wohin wurden auf einer Buchseite die Buchstaben gedruckt?* In Anlehnung an den Begriff des *distant reading* kann hier von *distant viewing* gesprochen werden.

Datengrundlage sind die XML-Daten aus dem OCR-Workflow der ersten Projektphase des DTA (2007–2010), die mit OCR erfasst, nachkorrigiert und in TEI P5 kodiert wurden. Es handelt sich um 199 Texte des *Kernkorpus*. Da das DTA seinen "Fokus", wie das Projektteam schreibt, auf die "Textdaten" legen möchte ("und die Verknüpfung zu den Faksimiledaten über die Seitenzuordnung ausreichend ist"), wurde dieser arbeitsaufwändige Workflow nicht mehr weiter geführt. Die entsprechenden Daten werden auf deren [Website](https://deutschestextarchiv.de/download#ocr) zur Verfügung gestellt. Die von mir am 30.1.2020 heruntergeladenen Daten datieren auf den 6.11.2013.

In den TEI-Daten des DTA werden jedem Buchstaben konkrete Koordinate nach folgendem Schema zugewiesen:

```xml
	<pb facs="#f0001"/>
	[…]
	<c ulx="44" uly="954" lrx="103" lry="1030" rendition="#c #b" xml:id="c1">R</c>
	<c ulx="102" uly="974" lrx="139" lry="1030" rendition="#c #b" xml:id="c2">u</c>
    <lb/>
	[…]
```

Ausgezeichnet sind in diesen Daten die TEI-*front*-, -*body*- und -*back*-Umgebungen (hier kommt nur der *body* in den Blick) und *framework*-Bereiche, das heißt beispielsweise Seitenzahlen, die hier nicht berücksichtigt werden.

Nota bene: Die Koordinaten beziehen sich weder auf die online dargestellten Faksimiles noch auf die Bilddaten der jeweiligen Urheber-Institution.

## Pakete

In [ ]:
#!/usr/bin/env python3
# -*- encoding: utf8 -*-

# Pakete importieren
import re, glob # Regex, Global
from bs4 import BeautifulSoup # XML Parser
from PIL import Image, ImageDraw # Python Imaging Library

## Auslesen der Daten aus den DTA-XML-Daten

In [ ]:
# Sicherheitsvorkehrung:
eingabe = input("Das starten dieses Abschnitts setzt einen großen Prozess in Gang und überschreibt womöglich bestehende Daten. Sicher? Y/N")
if eingabe == "N":
    import sys
    sys.exit()
elif eingabe == "Y":
    pass
else:
    import sys
    sys.exit()

# Programm:
filepaths = "dta_ocr_texte_2013-11-06/*.ready.xml"

for filepath in sorted(glob.glob(filepaths)):
    filename = filepath[25:]

    with open('Satzspiegel_DatenBilder/'+filename+'.csv', 'w', encoding='utf-8') as tempfh:
        tempfh.write('')
    
    # Filecontent
    with open(filepath, "r", encoding="utf-8") as fh:
        print("open file… " + filepath)
        bodysoup = BeautifulSoup(fh.read(), "lxml-xml").body # nur body-Bereich auswählen
        for fw in bodysoup.find_all("fw"): # Framework-Elemente entfernen
            fw.decompose()
        for c in bodysoup.find_all("c"): # c-Elemente finden
            if ("ulx" or "uly" or "lrx" or "lry") in c.attrs: # nur c-Elemente mit Koordinaten
                with open('Satzspiegel_DatenBilder/'+filename+'.csv', 'a', encoding='utf-8') as tempfh: # Koordinaten in Datei schreiben
                    tempfh.write( 
                        c['ulx']+','+
                        c['uly']+','+
                        c['lrx']+','+
                        c['lry']+'\n' )
print("Finished.")

## Zeichnen der ausgelesenen Daten

In [ ]:
filepaths = 'Satzspiegel_DatenBilder/*.csv'
for filepath in sorted(glob.glob(filepaths)):
    with open(filepath, 'r', encoding='utf-8') as fh:
        print("reading", filepath)
        file = fh.read()
        
        # Error counter for stats:
        errorc_x = 0
        errorc_y = 0
        
        im = Image.new('1', (3000, 3400), color=(1))
        draw = ImageDraw.Draw(im)
        for line in file.split("\n"):
            
            if line == "": # am Ende jeder Datei ist eine Leerzeile: diese überspringen
                continue
            
            coord = line.split(",") # Zeile splitten in Koordinaten

            ulx = int(coord[0])
            uly = int(coord[1])
            lrx = int(coord[2])
            lry = int(coord[3])
            
            #hier:
            if (ulx <= 0) or (ulx >= 3000) or (lrx <= 0) or (lrx >= 3000): # X-Koordinaten überprüfen / überspringen
                #print("- X-coord. out of window: ", coord)
                errorc_x += 1
                continue
            if (uly <= 0) or (uly >= 3400) or (lry <= 0) or (lry >= 3400): # Y-Koordinaten überprüfen / überspringen
                #print("- Y-coord. out of window: ", coord)
                errorc_y += 1
                continue

            try: # Zeichnen:
                draw.rectangle( (ulx, uly, lrx, lry), fill=(0))
            except: # Error-Handling:
                print("- error in file/line:", filepath, line)

        im.save(filepath+'.jpg', quality=50)
        print(" - error-counter (X/Y-coord.): "+
              str(errorc_x)+"/"+str(errorc_y)+", at "+str(len(file.split("\n")))+" chars")
print("Finished.")

## Weiterverarbeitung mit bash & imagemagick

Bilder runter rechnen, Montage und Animation erstellen:

```bash
cd Satzspiegel_DatenBilder
for file in *.jpg; do convert $file -resize 300x300 ${file##*/}; done
FILELIST=*.jpg
montage $FILELIST -tile 25x -border 1 -bordercolor black -geometry +0+1 montage.jpg
convert -delay 10 *. ../animation.gif
```

# Notizen zur Entscheidung, wieviel Darstellungsfläche gewählt werden soll

Die größte 'Fehlerquelle' (d.h. jene Werte, die entweder besonders häufig negativ sind oder unrealistisch hoch) sind die x-Koordinaten (_ulx_ und _lrx_). Siehe folgende Auswertung der *errorc_x* und *errorc_y* Ausgaben.

Hier, schmutzigerweise nur per `grep` (dh. inkl. allen c-Koordinaten, auch von *front*, *fw* usw.):

```bash
grep -Pho '(?<=uly=")(.+?)(?=")' dta_ocr_texte_2013-11-06/* > temp
sort temp | uniq -c | sort -nr | awk '{ print $2 "," $1}' > ulyvalues.csv
```

## zum ulx-Wert

Es gibt 8.746 negative ulx-Koordinaten (von '-9' bis '-1475', im Mittelwert 6 mal), exkl. der Koordinate **'-1'** mit **159.958** Vorkommen. (Einmal gibt es den Wert '-2147483648'.)

Es gibt insgesamt 56.372.345 positive Werte. Der häufigste positive Wert ist '816' (57.646 mal).

Der Wert '0' kommt 2.712 mal vor. Werte unter 15 gibt es selten (428 mal). Der häufigste Werte über 1.000 ist 1.001 (47.178 mal). Unter 1.000 ist der Mittelwert 805 (20.526.344 mal). Der häufigste Wert über 2.000 ist 2.002 (180 mal). Zwischen 2.000 und dem höchsten Wert 2.265 (außer der Koordinate '10000000') gibt es 22.665 Koordinaten.

Ein **Minimumwert von 15** schließt 561 und ein **Maximalwert von 2.200** schließt 841 Werte aus.

## zum lrx-Wert

Es gibt 8.782 negative lrx-Koordinaten ('-15' bis '-1466', im Mittelwert 6 mal), exkl. der Koordinate **'-1'** mit **116.614** Vorkommen. (Einmal gibt es den Wert '-2147483648'.)

Es gibt insgesamt 56.415.653 positive Werte. Der häufigste positive Wert ist '844' (57.613 mal).

Der Wert '0' kommt 656 mal vor (und weicht damit erheblich von der 0-Häufigkeit von ulx ab). Werte unter 30 gibt es selten (508 mal). Der häufigste Wert über 1.000 ist 1.001 (48.356 mal). Unter 1.000 ist der Mittelwert 819 (54.575 mal). Der häufigste Wert über 2.000 ist 2.003 (237 mal). Zwischen 2.000 und dem höchsten Wert 2.421 (außer der Koordinate '10000000') gibt es 26.487 Koordinaten.

Ein **Minimumwert von 30** schließt 508 und ein **Maximalwert von 2.200** schließt 1.780 Werte aus.

## Fazit x-Werte

Bei beiden Empfehlungen sind die erheblichen negativen Werte nicht berücksichtigt. Es gälte zu überprüfen, ob eine einfache Umkehrung des Vorzeichens, so dass die Werte positiv werden, eine realistische Möglichkeit ist, ob dieser Fehler bei der Vorverarbeitung geschehen sein kann.

## zum lry-Wert

Der Wert '-1' kommt sehr häufig (22.251 mal) vor. Als **Maximalwert** würde ich hier **2.500** nehmen, Minimum 0.

## zum uly-Wert

Der Wert -1 kommt sehr häufig (29.119) vor (hier auch 1.000.000 mit 85 mal und einmal 77.899). Als **Maximalwert** würde ich hier **3.000** nehmen, Minimum 0.

# FAZIT

Um sicher zu gehen und großräumig abzudecken, auch um mit einem tendenziell quadratischen Format kein Hochkant zu unterstützen wird das Ausgabeformat 3000 x 3400 gewählt. Werte unter 0 und über 3000 bzw. 3400 werden nicht berücksichtigt.